# Status
1. Currently the resolution is about 10% (166 electrons) by adding 7 chips.
2. Need to check:
   - Fano factor
   - recombination rate
   - TMS charge collection efficiency
   - If the ENC agree with the sum of noise from each chip
   - How to make the correction
   - Remove the data with changing gain
   
## Note
1. Why we think it's full $\alpha$ charge collection: the total electron is a good gaussian. If there is missing charge, it won't be a gussian, instead, a long tail on the left.
2. Where the bias is from: we are selecting around the largest point in each channel, considering the arrival time difference, but due to "look elsewhere effect", we can almost always find something provided that the search window is big enough. This causes a bias from the statistical fluctuation. 

In [1]:
#!/usr/bin/env python36

from rootUtil3 import waitRootCmdX, useNxStyle

import sys
sys.path.append('/home/dzhang/work/repos/TMSPlane/Software/Control/src')
from rootHelper import getRDF

from ROOT import gROOT, gInterpreter, RDF, gStyle, TLegend, gPad, TCanvas

useNxStyle()
gStyle.SetPalette(55)

dir1 = '/home/dzhang/work/repos/TMSPlane/Software/Control/src/data/fpgaLin/'

d1,ch1 = getRDF(dir1+'Mar08D1a/tpx01a_Mar08D1a_data_*.root', treename='reco')

xcode = '''
TFile* fin1 = new TFile("/data/repos/TMSPlane2/Software/Analysis/X19/scripts/calib_out.root","read");
auto* calib_gr_0 = (TGraphErrors*)fin1->Get("calib_gr_0");
auto* calib_gr_1 = (TGraphErrors*)fin1->Get("calib_gr_1");
auto* calib_gr_2 = (TGraphErrors*)fin1->Get("calib_gr_2");
auto* calib_gr_3 = (TGraphErrors*)fin1->Get("calib_gr_3");
auto* calib_gr_4 = (TGraphErrors*)fin1->Get("calib_gr_4");
auto* calib_gr_5 = (TGraphErrors*)fin1->Get("calib_gr_5");
auto* calib_gr_6 = (TGraphErrors*)fin1->Get("calib_gr_6");
auto* calib_gr_7 = (TGraphErrors*)fin1->Get("calib_gr_7");
auto* calib_gr_8 = (TGraphErrors*)fin1->Get("calib_gr_8");
auto* calib_gr_9 = (TGraphErrors*)fin1->Get("calib_gr_9");
auto* calib_gr_10 = (TGraphErrors*)fin1->Get("calib_gr_10");
auto* calib_gr_11 = (TGraphErrors*)fin1->Get("calib_gr_11");
auto* calib_gr_12 = (TGraphErrors*)fin1->Get("calib_gr_12");
auto* calib_gr_13 = (TGraphErrors*)fin1->Get("calib_gr_13");
auto* calib_gr_14 = (TGraphErrors*)fin1->Get("calib_gr_14");
auto* calib_gr_15 = (TGraphErrors*)fin1->Get("calib_gr_15");
auto* calib_gr_16 = (TGraphErrors*)fin1->Get("calib_gr_16");
auto* calib_gr_17 = (TGraphErrors*)fin1->Get("calib_gr_17");
auto* calib_gr_18 = (TGraphErrors*)fin1->Get("calib_gr_18");

const size_t nCh = 19;
std::vector< TGraphErrors* > calib_grs{calib_gr_0,calib_gr_1,calib_gr_2,calib_gr_3,calib_gr_4,calib_gr_5,calib_gr_6,calib_gr_7,calib_gr_8,calib_gr_9,calib_gr_10,calib_gr_11,calib_gr_12,calib_gr_13,calib_gr_14,calib_gr_15,calib_gr_16,calib_gr_17,calib_gr_18};

double getCorrVal(float a, int ich){return calib_grs[ich]->Eval(a);}
double getSum(double a0, double a1, double a2, double a3, double a4, double a5, double a6, double a7, double a8, double a9, double a10, double a11, double a12, double a13, double a14, double a15, double a16, double a17, double a18){return a0+a1+a3+a4+a5+a6+a7+a9+a10+a11+a12+a13+a14+a15+a16+a17+a18;}

double getSum2(double a0, double a1){return a0+a1;}
//double getSum2b(){return cQ0+cQ1;}
'''
gInterpreter.Declare(xcode)


d2 = d1
nCh = 19
for i in range(nCh):
    d2 = d2.Define('cQ{0}'.format(i),'getCorrVal(Q[{0:d}],{0:d})'.format(i))

excludeCh = [2,8]
# d2.Define('Etot','+'.join(['cQ{0:d}'.format(i) for i in range(nCh) if i not in excludeCh]))
# print('+'.join(['cQ{0:d}'.format(i) for i in range(nCh) if i not in excludeCh]))

d2 = d2.Define('sum2', 'getSum2(cQ0,cQ1)')

d2 = d2.Filter('&&'.join(['cQ{0:d}<10000'.format(i) for i in range(nCh)]))
d2 = d2.Filter('w2[19]<60')

# d2 = d2.Define('sum2', 'getSum2b()')
d2 = d2.Define('E', 'getSum('+','.join(['cQ{0:d}'.format(i) for i in range(nCh)])+')')

h0 = RDF.TH1DModel('h1','h1;E;N',100,-1000,6000)

gStyle.SetFuncColor(2)

Welcome to JupyROOT 6.16/00
Total entries: 192055


In [2]:
%jsroot on

In [3]:
h1 = d2.Histo1D(h0,'E')
h1.Draw()
h_cq0 = d2.Histo1D(h0,'cQ0')
h_cq0.SetLineColor(2)
h_cq0.GetValue().Draw('same')
gPad.Draw()

In [4]:
ht = d2.Histo2D(("h2d",'h2d;Total E;cQ0',100,-1000,6000,100,-1000,6000),'E','cQ0')
ht.Draw('colz')
gPad.SetRightMargin(0.15)
gPad.Draw()

In [5]:
gPad.SetRightMargin(0.15)
ht = d2.Histo2D(("h2d",'h2d;cQ1;cQ0',100,-1000,4000,100,-1000,5000),'cQ1','cQ0')
ht.Draw('colz')
gPad.Draw()

In [6]:
d_largeE = d2.Filter('E>4000')

In [7]:
ht = d_largeE.Histo1D(h0,'cQ0')
ht.Draw()
gPad.Draw()

In [8]:
ht = d_largeE.Histo2D(("h2d",'h2d',100,-1000,6000,100,-1000,6000),'E','cQ0')
ht.Draw('colz')
gPad.Draw()

In [9]:
ht = d_largeE.Histo2D(("h2d",'h2d',100,-1000,6000,100,-1000,6000),'E','cQ1')
ht.Draw('colz')
gPad.Draw()

In [10]:
ht = d_largeE.Histo2D(("h2d",'h2d',100,-1000,6000,100,-1000,6000),'E','cQ3')
ht.Draw('colz')
gPad.Draw()

In [11]:
ht = d_largeE.Histo2D(("h2d",'h2d',100,4000,6000,100,-1000,4000),'E','cQ15')
ht.Draw('colz')
gPad.Draw()

In [12]:
ht = d2.Histo1D(('ht','ht',100,-100,1000),'cQ15')
ht.Draw()
ht2 = d_largeE.Histo1D(('ht','ht',100,-100,1000),'cQ15')
ht2.Draw('same PLC')
gPad.Draw()

In [13]:
ht2 = d_largeE.Histo1D(('h2','h2',100,4000,6000),'E')
ht2.Draw()
ht2.GetValue().Fit('gaus',"","",4500,5500)
gPad.Draw()

 FCN=57.3577 FROM MIGRAD    STATUS=CONVERGED      69 CALLS          70 TOTAL
                     EDM=4.95924e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.14023e+01   1.39385e+00   3.96934e-03   5.50992e-04
   2  Mean         4.89406e+03   1.07447e+01   3.30394e-02   6.60220e-05
   3  Sigma        2.32823e+02   9.00585e+00   3.32960e-05   1.10992e-01


In [14]:
ht = d2.Histo2D(("h2d",'h2d',100,-1000,1000,100,-1000,1000),'cQ6','cQ9')
ht.Draw('colz')
gPad.Draw()

In [15]:
d_sel6 = d2.Filter('cQ6>300&&cQ9<100').Define('Ex','E-cQ9')

In [16]:
ht1 = d2.Histo1D(('h2','All',100,-1000,6000),'E')
ht2 = d_sel6.Histo1D(('h2','w/o Ch 9',100,-1000,6000),'Ex')
ht3 = d_sel6.Define('Q06','cQ0+cQ6+cQ5+cQ1+cQ17+cQ18+cQ16').Histo1D(('h2','7 chs',100,-1000,6000),'Q06')
ht1.Draw()
ht2.SetLineColor(2)
ht2.Draw('same')
ht3.SetLineColor(4)
ht3.Draw('same')

gPad.BuildLegend()
gPad.Draw()

In [17]:
## Ex: without chip 9 -- chip 2 and 8 are already removed
ht2 = d_sel6.Histo1D(('h2','h2',100,1000,5000),'Ex')
ht2.GetValue().Fit('gaus',"","",2000,3500)
fun1 = ht2.GetValue().GetFunction('gaus')
fun1.SetLineColor(2)
print(fun1.GetParameter(2)/fun1.GetParameter(1))
gPad.Draw()

0.08050054910790184
 FCN=336.497 FROM MIGRAD    STATUS=CONVERGED      69 CALLS          70 TOTAL
                     EDM=1.93927e-12    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     2.57184e+03   1.74211e+01   1.20206e-01   3.96016e-08
   2  Mean         2.84376e+03   1.20714e+00   1.07895e-02   9.89553e-07
   3  Sigma        2.28924e+02   1.00694e+00   9.97766e-06  -6.43840e-04


In [18]:
## Ex: 8 chips
ht2 = d_sel6.Define('Qv1','cQ0+cQ6+cQ5+cQ1+cQ17+cQ18+cQ16+cQ4').Histo1D(('h2','h2',100,1000,5000),'Qv1')
ht2.GetValue().Fit('gaus',"","",2000,3500)
fun1 = ht2.GetValue().GetFunction('gaus')
fun1.SetLineColor(2)
print(fun1.GetParameter(2)/fun1.GetParameter(1))
gPad.Draw()

0.06430485410713782
 FCN=587 FROM MIGRAD    STATUS=CONVERGED      68 CALLS          69 TOTAL
                     EDM=2.28247e-12    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.43228e+03   2.36762e+01   2.12352e-01  -3.57279e-08
   2  Mean         2.64813e+03   8.90795e-01   1.05412e-02  -1.25370e-06
   3  Sigma        1.70288e+02   7.67543e-01   1.30249e-05  -1.58017e-03


In [19]:
# 7 chips, chip4 further removed
ht2 = d_sel6.Define('Qv2','cQ0+cQ6+cQ5+cQ1+cQ17+cQ18+cQ16').Histo1D(('h2','h2',100,1000,5000),'Qv2')
ht2.GetValue().Fit('gaus',"","",2000,3500)
fun1 = ht2.GetValue().GetFunction('gaus')
fun1.SetLineColor(2)
print(fun1.GetParameter(2)/fun1.GetParameter(1))
gPad.Draw()

0.06461416455356064
 FCN=616.341 FROM MIGRAD    STATUS=CONVERGED      65 CALLS          66 TOTAL
                     EDM=5.5256e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.49776e+03   2.43152e+01   2.22063e-01   2.82562e-07
   2  Mean         2.57907e+03   8.77126e-01   1.05969e-02  -3.00989e-05
   3  Sigma        1.66644e+02   7.64859e-01   1.34737e-05  -6.40521e-02


In [20]:
# 7 chips, select only stable runs
ht2 = d_sel6.Filter('run>40').Define('Qv2','cQ0+cQ6+cQ5+cQ1+cQ17+cQ18+cQ16').Histo1D(('h2','h2',100,1000,5000),'Qv2')
ht2.GetValue().Fit('gaus',"","",2000,3500)
fun1 = ht2.GetValue().GetFunction('gaus')
fun1.SetLineColor(2)
print(fun1.GetParameter(2)/fun1.GetParameter(1))
gPad.Draw()

0.05824807029619799
 FCN=217.991 FROM MIGRAD    STATUS=CONVERGED      60 CALLS          61 TOTAL
                     EDM=3.75526e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     2.97851e+03   2.21239e+01   1.29156e-01   3.37261e-06
   2  Mean         2.55422e+03   9.06006e-01   6.45593e-03   5.06173e-05
   3  Sigma        1.48779e+02   6.52109e-01   8.27912e-06   6.11686e-03


In [21]:
# 5 chips -- further remove chip 5 and 16
ht2 = d_sel6.Filter('run>40&&cQ5<100&&cQ16<100').Define('Qv2t','cQ0+cQ6+cQ1+cQ17+cQ18').Histo1D(('h2','h2',100,1000,5000),'Qv2t')
ht2.GetValue().Fit('gaus',"","",2000,3500)
fun1 = ht2.GetValue().GetFunction('gaus')
fun1.SetLineColor(2)
print(fun1.GetParameter(2)/fun1.GetParameter(1))
gPad.Draw()

0.049409564444901063
 FCN=72.5657 FROM MIGRAD    STATUS=CONVERGED      59 CALLS          60 TOTAL
                     EDM=4.96117e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.18874e+03   1.55818e+01   5.27454e-02   8.59816e-07
   2  Mean         2.42083e+03   1.28845e+00   5.31490e-03  -1.92823e-06
   3  Sigma        1.19612e+02   9.25795e-01   8.48361e-06   1.45912e-02


In [22]:
# Another combination of 5 chips -- remove chip 1 and 18 -> low statistics, but similar results
ht2 = d_sel6.Filter('run>40&&cQ1<100&&cQ18<100').Define('Qv2f','cQ0+cQ6+cQ5+cQ17+cQ16').Histo1D(('h2','h2',100,1000,5000),'Qv2f')
ht2.GetValue().Fit('gaus',"","",2000,3500)
fun1 = ht2.GetValue().GetFunction('gaus')
fun1.SetLineColor(2)
print(fun1.GetParameter(2)/fun1.GetParameter(1))
gPad.Draw()

0.05184402488206654
 FCN=15.2657 FROM MIGRAD    STATUS=CONVERGED      56 CALLS          57 TOTAL
                     EDM=1.16235e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     6.08878e+01   3.50125e+00   5.58086e-03  -5.00727e-05
   2  Mean         2.34581e+03   5.79289e+00   1.14008e-02   7.70126e-05
   3  Sigma        1.21616e+02   4.29332e+00   1.81098e-05  -2.12885e-02


In [23]:
#Further remove chip 17, now only chip 0, 1, 6 and 18
ht2 = d_sel6.Filter('run>40&&cQ5<100&&cQ16<100&&cQ17<100').Define('Qv2z','cQ0+cQ6+cQ1+cQ18').Histo1D(('h2','h2',100,1000,3500),'Qv2z')
ht2.GetValue().Fit('gaus',"","",2000,3500)
fun1 = ht2.GetValue().GetFunction('gaus')
fun1.SetLineColor(2)
print(fun1.GetParameter(2)/fun1.GetParameter(1))
gPad.Draw()

0.046965529376392465
 FCN=100.055 FROM MIGRAD    STATUS=CONVERGED      59 CALLS          60 TOTAL
                     EDM=2.60796e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     7.77745e+02   1.01875e+01   4.07143e-02   2.97872e-06
   2  Mean         2.40116e+03   1.23184e+00   5.91302e-03   1.80223e-04
   3  Sigma        1.12772e+02   8.60453e-01   9.76805e-06  -9.98705e-04


In [24]:
ht2 = d_sel6.Histo1D(('h2','h2',100,-1000,3500),'cQ18')
ht2b = d_sel6.Filter('run>40&&cQ5<100&&cQ16<100&&cQ17<100').Histo1D(('h2','h2',100,-1000,3500),'cQ18')
ht2.Draw()
ht2b.SetLineColor(2)
ht2b.Draw('same')
gPad.Draw()

In [25]:
%jsroot on

In [26]:
#Further remove chip 17, now only chip 0, 1, 6 and 18
ht2 = d_sel6.Filter('run>40&&cQ5<100&&cQ16<100&&cQ17<100&&cQ18<100').Define('Qv2z1','cQ0+cQ6+cQ1').Histo1D(('h2','h2;N[e^{-}];Events',100,1000,3000),'Qv2z1')
ht2.GetValue().Fit('gaus',"","",2100,3000)
fun1 = ht2.GetValue().GetFunction('gaus')
fun1.SetLineColor(2)
print(fun1.GetParameter(2)/fun1.GetParameter(1))
gPad.Draw()

0.04257814338123833
 FCN=28.2127 FROM MIGRAD    STATUS=CONVERGED      59 CALLS          60 TOTAL
                     EDM=1.38381e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     4.41156e+02   7.49628e+00   1.57980e-02   3.33885e-05
   2  Mean         2.33175e+03   1.42305e+00   3.68764e-03  -5.14360e-07
   3  Sigma        9.92818e+01   1.09801e+00   7.77392e-06   1.70396e-01


In [27]:
### we can fit it with a crystal ball function
from ROOT import RooWorkspace, RooDataHist, RooArgList
w = RooWorkspace()
w.factory('Ns[5000,0,10000]*CBShape::model(x[1000,3000],mean[2332,2300,2350],sigma[100,80,120],alpha[2,0.1,5],n[1,0.1,10])')

x = w.var('x')
xframe = x.frame()
model = w.pdf('model')

#const char *name, const char *title, const RooArgList &vars, const TH1 *hist, Double_t initWgt=1.0)
args = RooArgList(x)
data = RooDataHist('data','data',args,ht2.GetValue())

#model.fitTo(data)

data.plotOn(xframe)
model.plotOn(xframe)
xframe.Draw()
gPad.Draw()


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

[#1] INFO:DataHandling -- RooDataHist::adjustBinning(data): fit range of variable x expanded to nearest bin boundaries: [1000,3000] --> [1000,3000]


## Check more

In [28]:
## Our dataset
d3 = d2.Filter('run>40&&E<4200')
ha1 = d3.Histo1D(('ha1','ha1;N_{e^{-}};Events',100,-800,4200),'E')
ha1.Draw()
gPad.Draw()

In [29]:
print(d3.Mean('E').GetValue())
means = [(v, d3.Mean('cQ'+str(v))) for v in range(19)]

for m in sorted(means, key=lambda x:x[1].GetValue()):
    print(m[0], m[1].GetValue())

2579.0251533490255
13 12.770953809801112
9 14.47156133409274
15 14.570038432505882
17 17.12961263628538
11 19.384474537754414
12 19.958561120617663
14 21.227054809178924
16 23.49217770521048
7 24.422400743443838
10 29.13658021888585
18 35.32463014813702
4 121.99031653840494
5 154.99232335773016
3 204.8482394731298
8 282.1797999110849
6 318.2771341655416
1 484.16789721425357
2 1043.85126832279
0 1062.8611971040293


In [84]:
ha1 = d3.Define('Et2','cQ0+cQ1+cQ6').Define('Et2v','E-Et2').Filter('cQ0>400&&cQ1>300&&cQ6>200&&cQ3<100&&cQ5<100&&cQ18<100').Histo1D(('ha1','ha1;N_{e^{-}};Events',100,-800,4200),'Et2')
ha1.Draw()
ha1.GetValue().Fit('gaus','','',2000,2600)
gPad.Draw()

 FCN=43.0751 FROM MIGRAD    STATUS=CONVERGED      60 CALLS          61 TOTAL
                     EDM=1.42234e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     9.48153e+02   1.52963e+01   3.93392e-02   1.63373e-05
   2  Mean         2.27474e+03   1.89540e+00   5.90367e-03  -2.68744e-04
   3  Sigma        1.31689e+02   1.52462e+00   1.00397e-05   8.57799e-04


 run | evt | comment
 ----| ---- | ---
 50 | 429 | A pulse and a signal

In [41]:
d3a = d3.Define('Et2','cQ0+cQ1+cQ6').Define('Et2v','E-Et2').Filter('cQ0>400&&cQ1>300&&cQ6>200&&cQ3<100&&cQ5<100&&cQ18<100')

import ROOT
v = ROOT.std.vector('std::string')()
for c in 'run','evt','tID':
    v.push_back(c)
pp1 = d3a.Filter('tID>3').Display(v)
#pp1 = d3a.Filter('tID>3').Display(('run','evt'))
pp1.Print()

run | evt | tID | 
56  | 643 | 4   | 
123 | 578 | 4   | 
98  | 26  | 5   | 
83  | 297 | 4   | 


In [32]:
pp1 = d3a.Filter('tID>3').Display("evt",20)
pp1.Print()

evt | 
643 | 
578 | 
26  | 
297 | 


In [33]:
pp1 = d3a.Filter('tID>3').Display("tID",20)
pp1.Print()

tID | 
4   | 
4   | 
5   | 
4   | 


In [42]:
print(means)

[(0, <ROOT.ROOT::RDF::RResultPtr<double> object at 0x13f8e960>), (1, <ROOT.ROOT::RDF::RResultPtr<double> object at 0x1407e360>), (2, <ROOT.ROOT::RDF::RResultPtr<double> object at 0x13c016a0>), (3, <ROOT.ROOT::RDF::RResultPtr<double> object at 0x13ce64f0>), (4, <ROOT.ROOT::RDF::RResultPtr<double> object at 0x13f194a0>), (5, <ROOT.ROOT::RDF::RResultPtr<double> object at 0x13fad870>), (6, <ROOT.ROOT::RDF::RResultPtr<double> object at 0xe40f430>), (7, <ROOT.ROOT::RDF::RResultPtr<double> object at 0x13b85460>), (8, <ROOT.ROOT::RDF::RResultPtr<double> object at 0x13a106d0>), (9, <ROOT.ROOT::RDF::RResultPtr<double> object at 0x12ef0e80>), (10, <ROOT.ROOT::RDF::RResultPtr<double> object at 0x13c29dc0>), (11, <ROOT.ROOT::RDF::RResultPtr<double> object at 0x13bf61b0>), (12, <ROOT.ROOT::RDF::RResultPtr<double> object at 0x13ba7ae0>), (13, <ROOT.ROOT::RDF::RResultPtr<double> object at 0x13ac16a0>), (14, <ROOT.ROOT::RDF::RResultPtr<double> object at 0x14037140>), (15, <ROOT.ROOT::RDF::RResultPtr<do

In [45]:
topList = [a[0] for a in sorted(means, key=lambda x:x[1].GetValue(),reverse=True) if a[0] not in [2,8]]
print(topList)
#ha1 = d3.Define('Et10','cQ0+cQ1+cQ6').Define('Et2v','E-Et2').Filter('cQ0>400&&cQ1>300&&cQ6>200&&cQ3<100&&cQ5<100&&cQ18<100').Histo1D(('ha1','ha1;N_{e^{-}};Events',100,-800,4200),'Et2')
#ha1.Draw()
#gPad.Draw()

[0, 1, 6, 3, 5, 4, 18, 10, 7, 16, 14, 12, 11, 17, 15, 9, 13]


In [63]:
def getCut(ic):
    return 'Etx{0:d}'.format(ic), '+'.join(['cQ{0:d}'.format(ix) for ix in topList[:ic]]), '&&'.join('cQ{0:d}<100'.format(ix) for ix in topList[ic:])

print(getCut(5))

('Etx5', 'cQ0+cQ1+cQ6+cQ3+cQ5', 'cQ4<100&&cQ18<100&&cQ10<100&&cQ7<100&&cQ16<100&&cQ14<100&&cQ12<100&&cQ11<100&&cQ17<100&&cQ15<100&&cQ9<100&&cQ13<100')


In [78]:
from ROOT import TLegend
histos = []
h0a = RDF.TH1DModel('h0a','h0a;E;N',100,0,4000)
for ii in range(1, 17):
    a = getCut(ii)
    print(a)
    h1 = d3.Filter(a[2]).Define(a[0], a[1]).Histo1D(RDF.TH1DModel('h0a',a[0]+';E;N',100,0,4000), a[0])
    histos.append(h1)

opt = ''
gStyle.SetPalette(51)
lg = TLegend(0.2,0.3,0.4,0.9)
for h1 in reversed(histos):
    h1.Draw('PLC'+opt)
    lg.AddEntry(h1.GetValue(),h1.GetValue().GetTitle(),"l")
    opt = 'same'
lg.Draw()
gPad.Draw()

('Etx1', 'cQ0', 'cQ1<100&&cQ6<100&&cQ3<100&&cQ5<100&&cQ4<100&&cQ18<100&&cQ10<100&&cQ7<100&&cQ16<100&&cQ14<100&&cQ12<100&&cQ11<100&&cQ17<100&&cQ15<100&&cQ9<100&&cQ13<100')
('Etx2', 'cQ0+cQ1', 'cQ6<100&&cQ3<100&&cQ5<100&&cQ4<100&&cQ18<100&&cQ10<100&&cQ7<100&&cQ16<100&&cQ14<100&&cQ12<100&&cQ11<100&&cQ17<100&&cQ15<100&&cQ9<100&&cQ13<100')
('Etx3', 'cQ0+cQ1+cQ6', 'cQ3<100&&cQ5<100&&cQ4<100&&cQ18<100&&cQ10<100&&cQ7<100&&cQ16<100&&cQ14<100&&cQ12<100&&cQ11<100&&cQ17<100&&cQ15<100&&cQ9<100&&cQ13<100')
('Etx4', 'cQ0+cQ1+cQ6+cQ3', 'cQ5<100&&cQ4<100&&cQ18<100&&cQ10<100&&cQ7<100&&cQ16<100&&cQ14<100&&cQ12<100&&cQ11<100&&cQ17<100&&cQ15<100&&cQ9<100&&cQ13<100')
('Etx5', 'cQ0+cQ1+cQ6+cQ3+cQ5', 'cQ4<100&&cQ18<100&&cQ10<100&&cQ7<100&&cQ16<100&&cQ14<100&&cQ12<100&&cQ11<100&&cQ17<100&&cQ15<100&&cQ9<100&&cQ13<100')
('Etx6', 'cQ0+cQ1+cQ6+cQ3+cQ5+cQ4', 'cQ18<100&&cQ10<100&&cQ7<100&&cQ16<100&&cQ14<100&&cQ12<100&&cQ11<100&&cQ17<100&&cQ15<100&&cQ9<100&&cQ13<100')
('Etx7', 'cQ0+cQ1+cQ6+cQ3+cQ5+cQ4+cQ18', 'cQ10<10

In [80]:
histos = []
h0b = RDF.TH1DModel('h0b','h0b;E;N',150,1000,4000)
for ii in range(1, 17):
    a = getCut(ii)
    h1 = d_sel6.Filter(a[2]).Define(a[0], a[1]).Histo1D(RDF.TH1DModel('h0b',a[0]+';E;N',150,1000,4000), a[0])
    histos.append(h1)

opt = ''
lg = TLegend(0.2,0.3,0.4,0.9)
for h1 in reversed(histos):
    h1.Draw('PLC'+opt)
    lg.AddEntry(h1.GetValue(),h1.GetValue().GetTitle(),"l")
    opt = 'same'
lg.Draw()
gPad.Draw()

In [82]:
for h1 in reversed(histos):
    print(h1.GetEntries(), h1.GetMean(), h1.GetStdDev())

37430.0 2829.420675689181 254.57144955854642
37430.0 2818.0970336108835 253.25738027358838
37206.0 2803.2513088497244 249.69561296597195
36851.0 2784.708207225701 247.09118581389214
35712.0 2765.921966836117 241.74858667231527
34884.0 2748.7111842525937 237.21457539680887
32143.0 2727.919056028582 226.5848960370954
29961.0 2699.984413566682 221.5868069344483
29231.0 2677.644217132735 221.07941908989335
26150.0 2655.106512711166 213.66630349063396
22351.0 2630.398596928008 222.37091502143412
17103.0 2554.351237687034 224.29553952723046
6640.0 2430.100794832936 239.59535508735837
5191.0 2373.12327231573 248.46106191438398
0.0 0.0 0.0
0.0 0.0 0.0


### Test

## Summary

- resolution 100e with 3 chips (number of electrons calculated from test pulse). Expect $\sqrt{3}\times 30e$
- Low yield. Large fraction of recombination? Low detection efficiency? Try increase the HV
- Check the expected number of electrons for alpha in Argon...


## Other

In [34]:
d_sel5 = d2.Filter('cQ5>700&&cQ9<100')
ht1 = d2.Histo1D(('h2','h2',100,-1000,6000),'E')
ht2 = d_sel5.Histo1D(('h2','h2',100,-1000,6000),'E')
ht3 = d_sel5.Define('Q05','cQ0+cQ6+cQ5+cQ4').Histo1D(('h2','h2',100,-1000,6000),'Q05')
ht1.Draw()
ht2.GetValue().SetLineColor(2)
ht2.Draw('same PCL')
ht3.GetValue().SetLineColor(4)
ht3.Draw('same PCL')
gPad.Draw()

In [35]:
ht2 = d_sel5.Define('Q05','cQ0+cQ6+cQ5+cQ4').Histo1D(('h2','h2',100,1000,5000),'Q05')
ht2.GetValue().Fit('gaus',"","",2000,3500)
fun1 = ht2.GetValue().GetFunction('gaus')
fun1.SetLineColor(2)
print(fun1.GetParameter(2)/fun1.GetParameter(1))
gPad.Draw()

0.05170510503537006
 FCN=156.175 FROM MIGRAD    STATUS=CONVERGED      66 CALLS          67 TOTAL
                     EDM=5.6094e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     2.93964e+02   8.68456e+00   3.67214e-02  -8.84544e-06
   2  Mean         2.52067e+03   2.73217e+00   1.62874e-02  -9.11886e-05
   3  Sigma        1.30332e+02   2.78435e+00   2.70870e-05  -3.44709e-02


In [36]:
ht2 = d_sel5.Histo1D(('h2','h2',100,1000,5000),'E')
ht2.GetValue().Fit('gaus',"","",2000,3500)
fun1 = ht2.GetValue().GetFunction('gaus')
fun1.SetLineColor(2)
print(fun1.GetParameter(2)/fun1.GetParameter(1))
gPad.Draw()

0.07154040415838947
 FCN=37.6744 FROM MIGRAD    STATUS=CONVERGED      60 CALLS          61 TOTAL
                     EDM=2.49076e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.89223e+02   4.87054e+00   1.14932e-02   1.86947e-05
   2  Mean         2.95925e+03   4.31513e+00   1.30714e-02   1.57261e-04
   3  Sigma        2.11706e+02   3.54595e+00   1.29460e-05   3.47620e-02


In [37]:
d_selv1v3 = d2.Filter('cQ5>700&&cQ1<50&&cQ3<50')
ht1 = d2.Histo1D(('h2','h2',100,-1000,6000),'E')
ht2 = d_selv1v3.Histo1D(('h2','h2',100,-1000,6000),'E')
ht3 = d_selv1v3.Define('Q05','cQ0+cQ6+cQ5+cQ4').Histo1D(('h2','h2',100,-1000,6000),'Q05')
ht1.Draw()
ht2.GetValue().SetLineColor(2)
ht2.Draw('same norm')
ht3.GetValue().SetLineColor(4)
ht3.Draw('same norm')
gPad.Draw()

In [38]:
ht2 = d_selv1v3.Histo1D(('h2','h2',100,1000,5000),'E')
ht2.GetValue().Fit('gaus',"","",2000,3500)
fun1 = ht2.GetValue().GetFunction('gaus')
fun1.SetLineColor(2)
print(fun1.GetParameter(2)/fun1.GetParameter(1))
gPad.Draw()

0.11773846913613234
 FCN=23.9633 FROM MIGRAD    STATUS=CONVERGED      84 CALLS          85 TOTAL
                     EDM=5.18568e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     4.74751e+00   8.16669e-01   1.25572e-03  -3.11089e-04
   2  Mean         2.79363e+03   4.68247e+01   1.08529e-01  -1.47714e-06
   3  Sigma        3.28918e+02   6.69791e+01   1.13242e-04   6.82045e-04


In [39]:
ch1.Show(0)

======> EVENT:0
 run             = 114
 evt             = 0
 tID             = 0
 Q               = 0.0398932, 
                  0.00182381, -9.05342e-05, 0.00596491, 0.00985983, 0.00367682, 
                  0.00275012, -0.000280219, -0.000378932, -0.000201427, 8.05693e-05, 
                  0.000673449, 0.000562031, 0.000748648, 0.000503591, 0.00174521, 
                  -0.0012886, 9.7149e-06, 0.000741042, 0.0347784
 im              = 211, 
                  160, 137, 212, 205, 206, 207, 85, 85, 331, 160, 
                  307, 159, 385, 289, 333, 85, 169, 340, 135
 w2              = 219, 
                  94, 0, 237, 216, 231, 137, 0, 0, 0, 27, 
                  85, 60, 54, 43, 170, 0, 4, 127, 40


In [40]:
ht2 = d_sel6.Filter('run>40&&cQ5<100&&cQ16<100&&cQ17<100&&cQ18<100').Define('Qv2z1','cQ0+cQ6+cQ1').Histo1D('tID')
ht2.Draw()
gPad.Draw()